In [ ]:
#|default_exp rag

# ShellSage Retrieval Augmented Generation

## Imports

In [ ]:
#| export
from chonkie import SentenceChunker
from fastcore.all import *
from lancedb import connect
from lancedb.pydantic import LanceModel, Vector
from pathlib import Path
from sentence_transformers import SentenceTransformer
from subprocess import check_output as co

import os, re, subprocess
os.environ['TOKENIZERS_PARALLELISM'] = 'false'

In [ ]:
import random

## Database setup

In [ ]:
#| export
# set up db path in user's home cache directory
db_path = Path.home() / '.cache' / 'shell_sage' / 'db'
db_path.mkdir(parents=True, exist_ok=True)

In [ ]:
#| export
chunker = SentenceChunker(tokenizer="gpt2", chunk_size=2_048,
                          chunk_overlap=256, min_sentences_per_chunk=1)

In [ ]:
#| export
model = SentenceTransformer('thenlper/gte-small')
ndim = model.encode(["Example sentence"]).shape[1]

In [ ]:
ndim

384

In [ ]:
#| export
class EmbeddingTable(LanceModel):
    content: str
    package_name: str
    embedding: Vector(ndim)

In [ ]:
#| export
db = connect(db_path)
tbl = None
try: tbl = db.open_table("man_pages")
except ValueError:
    tbl = db.create_table("man_pages", schema=EmbeddingTable, mode="create")
    tbl.create_fts_index("content") # for hybrid search

In [ ]:
db_path.ls()

(#1) [Path('/Users/nathan/.cache/shell_sage/db/man_pages.lance')]

## Man pages

In [ ]:
def _section(cmd, section):
    s = co(f'man {cmd} | col -b | sed -n "/^{section}/,/^[A-Z]/p" | sed "$d"',
              shell=True, stderr=subprocess.DEVNULL, text=True).strip()
    return '\n'.join(s.splitlines()[:-1]).strip()

In [ ]:
print(_section('ls', 'SYNOPSIS')[:128])
print(_section('ls', 'DESCRIPTION')[:128])

SYNOPSIS
     ls [-@ABCFGHILOPRSTUWabcdefghiklmnopqrstuvwxy1%,] [--color=when]
	[-D format] [file ...]
DESCRIPTION
     For each operand that names a file of a type other than directory, ls
     displays its name as well as any req


In [ ]:
def _get_page(cmd):
    name = _section(cmd, 'NAME')
    synopsis = _section(cmd, 'SYNOPSIS')
    description = _section(cmd, 'DESCRIPTION')
    examples = _section(cmd, 'EXAMPLES')
    return cmd, f"{name}\n\n{synopsis}\n\n{description}\n\n{examples}".strip()

In [ ]:
print(_get_page('ls')[1][:256])
print('...')
print(_get_page('ls')[1][-512:])

NAME
     ls – list directory contents

SYNOPSIS
     ls [-@ABCFGHILOPRSTUWabcdefghiklmnopqrstuvwxy1%,] [--color=when]
	[-D format] [file ...]

DESCRIPTION
     For each operand that names a file of a type other than directory, ls
     displays its name as
...
used in conjunction with the -l option.

EXAMPLES
     List the contents of the current working directory in long format:

	   $ ls -l

     In addition to listing the contents of the current working directory in
     long format, show inode numbers, file flags (see chflags(1)), and suffix
     each filename with a symbol representing its file type:

	   $ ls -lioF

     List the files in /var/log, sorting the output such that the most
     recently modified entries are printed first:

	   $ ls -lt /var/log


In [ ]:
def _manpages(lim=None):
    lines = L(co(['apropos', '-s', '1', '.'], text=True).strip().splitlines())
    lines = lines.map(lambda s: s.split("(")[0].strip())
    cmds = lines.filter(lambda s: s).unique()[:lim]
    pages = parallel(_get_page, cmds).filter(lambda x: x[1])
    return zip(*pages)

In [ ]:
cmds, pages = _manpages(lim=128)

In [ ]:
len(pages)

127

In [ ]:
rid = random.randint(0, len(pages))
cmd, page = cmds[rid], pages[rid]
print(cmd)
print(page[:128])
print('...')
print(page[-512:])

git-mktag
NAME
       git-mktag - Creates a tag object with extra validation

SYNOPSIS
       git mktag

DESCRIPTION
       Reads a tag co
...
 pass a git-fsck(1) check.

       The "fsck" check done mktag is stricter than what git-fsck(1) would run
       by default in that all fsck.<msg-id> messages are promoted from
       warnings to errors (so e.g. a missing "tagger" line is an error).

       Extra headers in the object are also an error under mktag, but ignored
       by git-fsck(1). This extra check can be turned off by setting the
       appropriate fsck.<msg-id> varible:

	   git -c fsck.extraHeaderEntry=ignore mktag <my-tag-with-headers


## Indexing

In [ ]:
#| export
def index_man_pages(cmds, pages):
    for cmd, chunks in zip(cmds, chunker.chunk_batch(pages)):
        embds = model.encode([c.text for c in chunks])
        tbl.add([EmbeddingTable(content=c.text, package_name=cmd, embedding=emb)
                 for c, emb in zip(chunks, embds)])

In [ ]:
index_man_pages(cmds, pages)
len(tbl)

🦛 choooooooooooooooooooonk 100% • 127/127 docs chunked [00:00<00:00, 767.83doc/s] 🌱


In [ ]:
#| export
def search(q: str, limit: int=2, threshold: float=0.5):
    q_emb = model.encode([q])
    df = tbl.search(q_emb).metric("cosine").limit(limit).to_pandas()
    df = df.rename(columns={"_distance": "cosine_distance"})
    df = df[df.cosine_distance < 1 - threshold]
    return df

In [ ]:
df = search("How can I change my current branch?")
df.head()

,content,package_name,embedding,cosine_distance
0,\t Note the quotes around *.c. The file hell...,git-checkout,"[-0.041774247, -0.044317152, 0.0674934, 0.0194...",0.152378
1,NAME\n git-checkout - Switch branches or...,git-checkout,"[-0.048017204, -0.049874607, 0.05528505, -0.00...",0.153252


In [ ]:
doc = df.iloc[1]
print(doc.content[:512])
print(doc.package_name)

NAME
       git-checkout - Switch branches or restore working tree files

SYNOPSIS
       git checkout [-q] [-f] [-m] [<branch>]
       git checkout [-q] [-f] [-m] --detach [<branch>]
       git checkout [-q] [-f] [-m] [--detach] <commit>
       git checkout [-q] [-f] [-m] [[-b|-B|--orphan] <new-branch>] [<start-point>]
       git checkout [-f|--ours|--theirs|-m|--conflict=<style>] [<tree-ish>] [--] <pathspec>...
       git checkout [-f|--ours|--theirs|-m|--conflict=<style>] [<tree-ish>] --pathspec-from-fil
git-checkout


## CLI

In [ ]:
#| export
@call_parse
def main():
    "Index man pages for RAG"
    cmds, pages = _manpages()
    index_man_pages(cmds, pages)